In [1]:
import pandas as pd
import numpy as np
import math
import random
pd.set_option('display.max_rows', 500)

In [2]:
master_data = pd.read_csv("../../Data/filtered/master-data.csv", sep=",", encoding='Latin-1', low_memory=False)

In [3]:
master_data.head()

,compensation,year,country,age,gender,IT_experience_in_years,company_size,occupation,proficient_languages,desktop_OS,job_satisfaction,GDP
0,20000-40000,2011,United States,18_to_24,male,41310,below_100,"Executive (VP of Eng, CTO, CIO, etc.)","['JavaScript', 'CSS', 'PHP', 'Python', 'SQL', ...",Windows 7,5.0,1.559973e+13
1,80000-100000,2011,United States,18_to_24,male,41435,1000_to_4999,Web Application Developer,"['JavaScript', 'CSS', 'PHP', 'SQL']",Linux,6.0,1.559973e+13
2,80000-100000,2011,Germany,35_to_44,male,11,500_to_999,Desktop Application Developer,['C#'],Windows 7,6.0,3.749315e+12
3,60000-80000,2011,United Kingdom,35_to_44,male,11,500_to_999,IT Staff / System Administrator,"['SQL', 'C#', 'C++', 'C']",Windows 7,6.0,2.674891e+12
4,40000-60000,2011,United Kingdom,25_to_34,female,11,below_100,Web Application Developer,"['JavaScript', 'Python', 'SQL', 'C++', 'C', 'P...",Linux,5.0,2.674891e+12


In [4]:
#begin replacing values 
master_data['company_size'] = master_data['company_size'].replace(['Start Up (1-25)',
 'Mature Small Business (25-100)',
 'Mid Sized (100-999)',
 'Student',
 'Fortune 1000 (1,000+)',
 'nan',
 'Other (not working, consultant, etc.)',
 '101-999',
 '26-100',
 '1,000-3,000',
 '1/25/2013',
 '3001',
 '100',
 '1/5/2014',
 '50-100',
 '6/15/2014',
 '16-30',
 '31-50',
 'unknown',
 '100-499 employees',
 'I am not part of a company',
 '10-19 employees',
 '5-9 employees',
 '20-99 employees',
 '1-4 employees',
 '500-999 employees',
 '1,000-4,999 employees',
 '10,000+ employees',
 '5,000-9,999 employees',
 'I am not sure',
 'Other (please specify)',
 '20 to 99 employees',
 '10,000 or more employees',
 '10 to 19 employees',
 'Fewer than 10 employees',
 '5,000 to 9,999 employees',
 '100 to 499 employees',
 '1,000 to 4,999 employees',
 '500 to 999 employees',
 "I don't know",
 'I prefer not to answer',
 'Just me - I am a freelancer, sole proprietor, etc.',
 '2-9 employees',
 '2 to 9 employees',
 'I donÃ\x83Â¢Ã\x82Â\x80Ã\x82Â\x99t know', 'nan'],
    ['< 100',
'< 100',
'500 - 999',
'Student',
'1,000 - 4,999',
'nan',
'Other',
'500 - 999',
'< 100',
'1,000 - 4,999',
'nan',
'1,000 - 4,999',
'100 - 499',
'nan',
'< 100',
'nan',
'< 100',
'< 100',
'Unknown',
'100 - 499',
'Other',
'< 100',
'< 100',
'< 100',
'< 100',
'500 - 999',
'1,000 - 4,999',
'10,000 +',
'5,000 - 9,999',
'Unknown',
'Other',
'< 100',
'10,000 +',
'< 100',
'< 100',
'5,000 - 9,999',
'100 - 499',
'1,000 - 4,999',
'500 - 999',
'Unknown',
'Unknown',
'Other',
'Unknown',
'< 100',
'Unknown', 'nan'])


In [5]:
#preview the changes made for company size
master_data.head()
#preview unique list of grouping for company size
list(master_data["company_size"].unique())

['below_100',
 '1000_to_4999',
 '500_to_999',
 'Unknown',
 '100_to_499',
 'above_10000',
 '5000_to_9999']

In [6]:
master_data.groupby(["year", "company_size"])["company_size"].count()

year  company_size
2011  1000_to_4999      471
      500_to_999        378
      Unknown            79
      below_100        1112
2012  1000_to_4999      723
      500_to_999        757
      Unknown           106
      below_100        1939
2013  1000_to_4999     1249
      500_to_999       1078
      Unknown          1774
      below_100        1046
2014  100_to_499       1176
      Unknown          2255
      below_100        1174
2015  Unknown         14556
2016  1000_to_4999     3274
      100_to_499       6152
      5000_to_9999     1238
      500_to_999       2079
      Unknown          6996
      above_10000      4239
      below_100       17325
2017  1000_to_4999     1285
      100_to_499       2604
      5000_to_9999      519
      500_to_999        849
      Unknown           212
      above_10000      1822
      below_100        5562
2018  1000_to_4999     4264
      100_to_499       7691
      5000_to_9999     1661
      500_to_999       2531
      Unknown          6206
 

In [7]:
#one more clean up step for company size, notice the last 500-999 does not have the spaces
def normalise_company_size(row):
    company_size = row 
    if row in ["< 100"]:
        company_size= "below_100"
    elif row in ["100 - 499"]:
        company_size= "100_to_499"
    elif row in ["500 - 999"]:
        company_size= "500_to_999"
    elif row in ["1,000 - 4,999"]:
        company_size= "1000_to_4999"
    elif row in ["5,000 - 9,999"]:
        company_size= "5000_to_9999"
    elif row in ["10,000 +"]:
        company_size="above_10000"
    elif ("I don" in row) or (row in ["Other", "nan"]):
        company_size= "Unknown"
    return company_size

In [8]:
master_data["company_size"] = master_data["company_size"].fillna("Unknown")
master_data["company_size"] = master_data["company_size"].apply(normalise_company_size)

In [9]:
# Dropping the records by students 
master_data = master_data[master_data.company_size != 'Student']

In [10]:
# Final list
master_data["company_size"].unique().tolist()

['below_100',
 '1000_to_4999',
 '500_to_999',
 'Unknown',
 '100_to_499',
 'above_10000',
 '5000_to_9999']

In [11]:
# Write the normalised company size to master data

(
    master_data.to_csv("../../Data/filtered/master-data.csv", index=False)
)
